In [1]:
pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)


Mounted at /content/drive/


In [3]:
import os
import librosa
from tqdm import tqdm
from glob import glob
import pandas as pd
import numpy as np

In [4]:
from jiwer import wer, cer

In [6]:
sylhet_main = pd.read_excel('/content/drive/MyDrive/Sylhet_Audio/Sylhet_Data.xlsx')
sylhet_main

,Unnamed: 0,Original,External_ID,Annotator,Contents,Time,District
0,2.0,rec_01.wav,rec_01_audio_1.wav,shah89@student.sust.edu,"<> ডিউটি খরি, ডিউটি টু বাড়িত। আর তো <> আর সবকি...",161.0,Sylhet
1,4.0,rec_01.wav,rec_01_audio_3.wav,dipa.sust44@gmail.com,"করিয়া তারপরে রেস্ট নিলাম কিছুক্ষন, রেস্ট নেওয়া...",412.0,Sylhet
2,5.0,rec_01.wav,rec_01_audio_4.wav,dipa.sust44@gmail.com,আট ঘন্টার ডিউটি পরে প্রতিদিনই। এইরকম ভাবে চেঞ্...,356.0,Sylhet
3,6.0,rec_01.wav,rec_01_audio_5.wav,dipa.sust44@gmail.com,"কিরকম লাগে আমরা ছাত্র-ছাত্রীরা, স্যাররা? অনেক ...",979.0,Sylhet
4,7.0,rec_01.wav,rec_01_audio_6.wav,dipa.sust44@gmail.com,"ডিউটিটা করতে খুব ভালা লাগে, সবকিছু মিলায়া খুব ...",675.0,Sylhet
...,...,...,...,...,...,...,...
617,619.0,rec_45.wav,rec_45_big_audio_2_chunk_0.wav,shah89@student.sust.edu,"আস্তে, আস্তে তো ওলাখান যার, যার। ফরে এখদিন এখ ...",509.0,Sylhet
618,620.0,rec_45.wav,rec_45_big_audio_5.wav,shah89@student.sust.edu,"ফড়ার খেত্রে তখন তারারে ডায়রি লেখিয়া দেওয়া অয়, ...",347.0,Sylhet
619,621.0,rec_45.wav,rec_45_big_audio_6.wav,shah89@student.sust.edu,"চেষ্টা খরি, ইসময় ফয়লা ফরীক্কা। ইটার ফরীক্কা আম...",774.0,Sylhet
620,622.0,rec_45.wav,rec_45_big_audio_8_chunk_0.wav,shah89@student.sust.edu,নায়কর বাফে ওইলো গিয়া ইসময় জমিদারোর সেরেস্তা কি...,458.0,Sylhet


In [7]:
sylhet_main.columns

Index(['Unnamed: 0', 'Original', 'External_ID', 'Annotator', 'Contents',
       'Time', 'District'],
      dtype='object')

In [8]:
sylhet_main = sylhet_main.drop(labels = ['Unnamed: 0','Original','Annotator','District','Time'], axis = 1)
sylhet_main

,External_ID,Contents
0,rec_01_audio_1.wav,"<> ডিউটি খরি, ডিউটি টু বাড়িত। আর তো <> আর সবকি..."
1,rec_01_audio_3.wav,"করিয়া তারপরে রেস্ট নিলাম কিছুক্ষন, রেস্ট নেওয়া..."
2,rec_01_audio_4.wav,আট ঘন্টার ডিউটি পরে প্রতিদিনই। এইরকম ভাবে চেঞ্...
3,rec_01_audio_5.wav,"কিরকম লাগে আমরা ছাত্র-ছাত্রীরা, স্যাররা? অনেক ..."
4,rec_01_audio_6.wav,"ডিউটিটা করতে খুব ভালা লাগে, সবকিছু মিলায়া খুব ..."
...,...,...
617,rec_45_big_audio_2_chunk_0.wav,"আস্তে, আস্তে তো ওলাখান যার, যার। ফরে এখদিন এখ ..."
618,rec_45_big_audio_5.wav,"ফড়ার খেত্রে তখন তারারে ডায়রি লেখিয়া দেওয়া অয়, ..."
619,rec_45_big_audio_6.wav,"চেষ্টা খরি, ইসময় ফয়লা ফরীক্কা। ইটার ফরীক্কা আম..."
620,rec_45_big_audio_8_chunk_0.wav,নায়কর বাফে ওইলো গিয়া ইসময় জমিদারোর সেরেস্তা কি...


In [9]:
len(sylhet_main)

622

In [11]:
sylhet_b_gACR = pd.read_excel('/content/drive/MyDrive/Sylhet_Audio/Google_ASR/sylhet-googleASR-inferences.xlsx')
sylhet_b_gACR

,Unnamed: 0,External_ID,Contents
0,0,rec_01_audio_3.wav,ফরিয়া তারপরে বিকাল টাইমে উঠলাম নামাজ টামাজ পড...
1,1,rec_01_audio_0.wav,শুনছেন আপনার বাসার সবাই বা দেশের কি রকম পরিস্থ...
2,2,rec_01_audio_1(1).wav,ভিডিও গাড়ি ডিউটি টু বাড়ির সবকিছু ভালো আছে সব...
3,3,rec_01_audio_1.wav,ভিডিও গাড়ি ডিউটি টু বাড়ির সবকিছু ভালো আছে সব...
4,4,rec_01_audio_2.wav,কেন আমি কোন বালা কিভাবে ডিউটি ডিউটি করবেন
...,...,...,...
617,617,rec_45_big_audio_1_chunk_0.wav,<>
618,618,rec_45_big_audio_13_chunk_1.wav,<>
619,619,rec_45_big_audio_2_chunk_0.wav,<>
620,620,rec_45_big_audio_11_chunk_0.wav,<>


In [12]:
len(sylhet_b_gACR)

622

In [14]:
def make_equal(dataset):

    df1 = sylhet_main
    df2 = dataset

    df = df2.join(df1.set_index('External_ID'), on='External_ID', how = 'left')
    df = df.drop(columns=["Contents"], axis = 1)
    df['Contents'] = df['Contents'].fillna("Null")
    return df


def errors(dataset, dname):

    dName = str(dataset)

    if len(dataset) == len(sylhet_main):

        CER = []
        WER = []

        dataset['Contents'] = dataset['Contents'].replace(["<>"], "")

        k = 1
        for i, j in zip(sylhet_main['Contents'],dataset['Contents']):
            # print("{} -->", k)
            # print("main: {}",i)
            # print("\n")
            # print("inference: {}",j)
            # print("**************************\n")
            WER.append(wer(str(i), str(j)))
            k += 1


        for i, j in zip(sylhet_main['Contents'],dataset['Contents']):
            CER.append(cer(str(i), str(j)))


        sylhet_main[f"CER ({dname})"] = CER
        sylhet_main[f"WER ({dname})"] = WER

    else:
        print("In\n")
        dataset = make_equal(dataset)
        errors(dataset,dname)



    return sylhet_main

In [15]:
df = errors(sylhet_b_gACR , 'g_ACR')
df

,External_ID,Contents,CER (g_ACR),WER (g_ACR)
0,rec_01_audio_1.wav,"<> ডিউটি খরি, ডিউটি টু বাড়িত। আর তো <> আর সবকি...",0.806897,1.000000
1,rec_01_audio_3.wav,"করিয়া তারপরে রেস্ট নিলাম কিছুক্ষন, রেস্ট নেওয়া...",0.816216,1.000000
2,rec_01_audio_4.wav,আট ঘন্টার ডিউটি পরে প্রতিদিনই। এইরকম ভাবে চেঞ্...,0.816456,0.965517
3,rec_01_audio_5.wav,"কিরকম লাগে আমরা ছাত্র-ছাত্রীরা, স্যাররা? অনেক ...",0.821990,0.968750
4,rec_01_audio_6.wav,"ডিউটিটা করতে খুব ভালা লাগে, সবকিছু মিলায়া খুব ...",0.830882,1.000000
...,...,...,...,...
617,rec_45_big_audio_2_chunk_0.wav,"আস্তে, আস্তে তো ওলাখান যার, যার। ফরে এখদিন এখ ...",1.000000,1.000000
618,rec_45_big_audio_5.wav,"ফড়ার খেত্রে তখন তারারে ডায়রি লেখিয়া দেওয়া অয়, ...",1.000000,1.000000
619,rec_45_big_audio_6.wav,"চেষ্টা খরি, ইসময় ফয়লা ফরীক্কা। ইটার ফরীক্কা আম...",1.000000,1.000000
620,rec_45_big_audio_8_chunk_0.wav,নায়কর বাফে ওইলো গিয়া ইসময় জমিদারোর সেরেস্তা কি...,1.000000,1.000000


In [16]:
metrics = {}


Avg_CER_gACR  = np.mean(df["CER (g_ACR)"])
Avg_WER_gACR = np.mean(df["WER (g_ACR)"])

metrics["g_ACR"] = f"Avg_WER : {Avg_WER_gACR} | Avg_CER : {Avg_CER_gACR }"

metrics

{'g_ACR': 'Avg_WER : 1.1684841591827337 | Avg_CER : 1.1795722678827878'}